In [20]:
import pandas as pd
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas_gbq import to_gbq
import wget
from zipfile import ZipFile

### Url de dowload dos Formulários Cadastrais de Companhias Abertas (FCA)

In [21]:
url_cvm = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/'

### Carregar Credencial do Google Big Query(GBQ)

In [3]:
#Carrega o arquivo json com a chave de autenticação do GBQ
credencial = service_account.Credentials.from_service_account_file(
    r'C:\Users\usuario\Documents\gbq.json',
    scopes=['https://www.googleapis.com/auth/bigquery']
)

### Criar lista com nome dos arquivos das DFCs

In [22]:
arquivos_zip = []

#Preencher a Lista para os anos de 2019 a 2023
for ano in range(2019, 2024):
    #Adiciona cada arquivo zip a lista criada
    arquivos_zip.append(f'fca_cia_aberta_{ano}.zip')
arquivos_zip

['fca_cia_aberta_2019.zip',
 'fca_cia_aberta_2020.zip',
 'fca_cia_aberta_2021.zip',
 'fca_cia_aberta_2022.zip',
 'fca_cia_aberta_2023.zip']

### Fazer dowload de todos os arquivos
• Verifica se já existe um arquivo com o nome e apaga e substitui pelo novo fazendo dowload novamente

In [24]:
for arquivo in arquivos_zip:
    #Caminho para salvar cada arquivo
    caminho_arquivo_zip = f'Cadastro_Zip/{arquivo}'
    #Caso já exista um arquivo com mesmo nome, é apagado
    if os.path.exists(caminho_arquivo_zip):
        os.remove(caminho_arquivo_zip)

    #Faz o dowload de cada arquivo zip de cada anos das DFC dentro da pasta Demonstracoes_Zip
    wget.download(url_cvm + arquivo, caminho_arquivo_zip)

### Extrair arquivos da pasta zipada

In [25]:
for arquivo in arquivos_zip:
    #Extrair os arquivos Zip das DFC nas pasta Demonstracoes
    ZipFile(f'Cadastro_Zip/{arquivo}', 'r').extractall('Cadastro')

### Carrega a lista dos arquivos do diretório Demonstracoes

In [26]:
#Diretorio do arquivo atual
diretorio_atual = os.getcwd()
caminho_diretorio = os.path.join(diretorio_atual, 'Cadastro')


arquivos_dfc = os.listdir('Cadastro/')

print(arquivos_dfc)

['fca_cia_aberta_2019.csv', 'fca_cia_aberta_2020.csv', 'fca_cia_aberta_2021.csv', 'fca_cia_aberta_2022.csv', 'fca_cia_aberta_2023.csv', 'fca_cia_aberta_auditor_2019.csv', 'fca_cia_aberta_auditor_2020.csv', 'fca_cia_aberta_auditor_2021.csv', 'fca_cia_aberta_auditor_2022.csv', 'fca_cia_aberta_auditor_2023.csv', 'fca_cia_aberta_canal_divulgacao_2019.csv', 'fca_cia_aberta_canal_divulgacao_2020.csv', 'fca_cia_aberta_canal_divulgacao_2021.csv', 'fca_cia_aberta_canal_divulgacao_2022.csv', 'fca_cia_aberta_canal_divulgacao_2023.csv', 'fca_cia_aberta_departamento_acionistas_2019.csv', 'fca_cia_aberta_departamento_acionistas_2020.csv', 'fca_cia_aberta_departamento_acionistas_2021.csv', 'fca_cia_aberta_departamento_acionistas_2022.csv', 'fca_cia_aberta_departamento_acionistas_2023.csv', 'fca_cia_aberta_dri_2019.csv', 'fca_cia_aberta_dri_2020.csv', 'fca_cia_aberta_dri_2021.csv', 'fca_cia_aberta_dri_2022.csv', 'fca_cia_aberta_dri_2023.csv', 'fca_cia_aberta_endereco_2019.csv', 'fca_cia_aberta_enderec

### Filtrar arquivos para carga no GBQ
É filtrado os arquivos que contém informações cadastrais das empresas e utilizado apenas a coluna de CNPJ que funcionará como chave unica e a coluna do setor de atividade

In [27]:
#Cria um DataFrame(DF) vazio
df = pd.DataFrame()

for arquivo in arquivos_dfc:
    #Filtrar arquivos a serem desconsiderados, arquivo de dados cadastrais sempre terá 23 digitos
    if  'geral' not in arquivo.lower():
        pass
    else:
        #Passa o caminho de cada arquivo a ser análisado na pasta Demonstracoes
        caminho_arquivo = os.path.join(caminho_diretorio, arquivo)
        #Gera um DF de cada arquivo do diretório Demonstracoes
        df_arquivo = pd.read_csv(caminho_arquivo, sep=';', decimal=',', encoding='ISO-8859-1')
        #Adiciona cada arquivo dentro do DF vazio
        df = pd.concat([df, df_arquivo], ignore_index=True)

#Deixa apenas as colunas de CNPJ e Setor de Atividade
df = df[['CNPJ_Companhia', 'Setor_Atividade']]
#Remove valores duplicados, pois como é utilizado o arquivo de varios anos a maioria das empresas terá informações duplicadas
df = df.drop_duplicates(subset='CNPJ_Companhia')
print(df.head())


       CNPJ_Companhia                        Setor_Atividade
0  00.000.000/0001-91                                 Bancos
1  00.000.208/0001-00                                 Bancos
2  00.001.180/0001-26                       Energia Elétrica
3  00.070.698/0001-11                       Energia Elétrica
4  00.185.475/0001-08  Emp. Adm. Part. - Sem Setor Principal


### Apagar dados da tabela raw_dfc
É executado consulta sql no GBQ para apagar os dados da tabela para não duplicar caso já tenha sido feita uma carga com os dados anteriormente

In [28]:
#Instanciar o cliente do GBQ
cliente = bigquery.Client(credentials=credencial, project=credencial.project_id)

#Query pra apagar os dados da tabela raw_dfc onde será carregado os dados das DFC
sql_apagar_dados = """ 
TRUNCATE TABLE `prjeto.dfc.raw_fca` 
"""

#Executar a query de apagar os dados da tabela
query = cliente.query(sql_apagar_dados)




### Inserir dados dentro da tabela raw_dfc
• É realizado a inserção de todos os dados do DF consolidados das DFC dentro da tabela raw_dfc no GBQ

In [29]:
#Inserir os dados do DF consolidado das DFC na tabela raw_dfc do GBQ
df.to_gbq(
    destination_table='prjeto.dfc.raw_fca',
    project_id='prjeto',
    if_exists='replace',
    credentials=credencial)

100%|██████████| 1/1 [00:00<00:00, 1000.07it/s]


### Consulta para verificar se foram carregados os dados na tabela

In [30]:

#Query pra apagar os dados da tabela raw_dfc onde será carregado os dados das DFC
sql_selecionar_dados = """ 
SELECT * FROM `prjeto.dfc.raw_fca` LIMIT 10
"""

#Executar a query de apagar os dados da tabela
query = cliente.query(sql_selecionar_dados)

# Coletar os resultados
results = query.result()

# Exibir os resultados
for row in results:
    print(row)


Row(('61.231.155/0001-23', 'Pesca'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('00.000.000/0001-91', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('00.000.208/0001-00', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('00.416.968/0001-01', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('01.548.981/0001-79', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('04.902.979/0001-44', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('04.913.711/0001-08', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('07.237.373/0001-20', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('08.884.213/0001-35', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
Row(('13.009.717/0001-46', 'Bancos'), {'CNPJ_Companhia': 0, 'Setor_Atividade': 1})
